In [151]:
import numpy as np
import pandas as pd
import scipy.stats
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import functools
 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from lightgbm import LGBMRegressor,LGBMClassifier
from sklearn.model_selection import cross_val_predict, GridSearchCV, KFold

# load data

In [177]:
data_x = pd.read_csv("./data/practice/acic_practice_0001.csv")
data_y = pd.read_csv("./data/practice_year/acic_practice_year_0001.csv")
data_y = pd.concat([data_y[data_y['year'] == i] for i in range(1,5)])

ct = ['X2','X4']
for c in ct:
    data_x[c] = data_x[c].astype('category')
    
X = ['X1','X2','X3','X4','X5','X6','X7','X8','X9']
V = ['V1_avg', 'V2_avg', 'V3_avg', 'V4_avg', 'V5_A_avg', 'V5_B_avg', 'V5_C_avg']
XV = V + X
Y = ['Y']
T = ['Z']

In [178]:
data_x.columns

Index(['id.practice', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9'], dtype='object')

In [165]:
data_y.columns

Index(['id.practice', 'year', 'Y', 'Z', 'post', 'n.patients', 'V1_avg',
       'V2_avg', 'V3_avg', 'V4_avg', 'V5_A_avg', 'V5_B_avg', 'V5_C_avg'],
      dtype='object')

# create 10-fold indices

In [179]:
kf = KFold(n_splits=10, shuffle=True, random_state=1)
indices = []
for train_index, test_index in kf.split(data_x):
    #print("TRAIN:", train_index, "TEST:", test_index)
    indices.append((train_index, test_index))

In [111]:
for train_index, test_index in indices:
    print(len(train_index), len(test_index))

450 50
450 50
450 50
450 50
450 50
450 50
450 50
450 50
450 50
450 50


# cross-fitting and estimate ATTE

In [180]:
# ML method objects, to estimate e(X) and g_0(V,X)
prob_m = LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         #metric = 'log_loss',
                         metric = 'binary_logloss,auc',
                         max_depth = 8,
                         num_leaves = 20,
                         learning_rate = 0.1,
                         #feature_fraction = 0.7,
                         min_child_samples=20,
                         min_child_weight=0.001,
                         #bagging = 1,
                         #subsample_freq = 2,
                         reg_alpha = 0.002,
                         reg_lambda = 10,
                         cat_smooth = 0,
                         n_estimators = 200,   
                        )

mean_m = LGBMRegressor(is_unbalance = True,
                       #metric = 'log_loss',
                       metric = 'l2',
                       max_depth = -1,
                       num_leaves = 20,
                       learning_rate = 0.1,
                       #feature_fraction = 0.7,
                       min_child_samples=20,
                       min_child_weight=0.001,
                       #bagging = 1,
                       #subsample_freq = 2,
                       reg_alpha = 0.002,
                       reg_lambda = 10,
                       cat_smooth = 0,
                       n_estimators = 200,   
                       )

In [181]:
data_x.columns

Index(['id.practice', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9'], dtype='object')

In [197]:
data_x.columns

Index(['id.practice', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9',
       'ps'],
      dtype='object')

# propensity score

In [196]:
# propensity score
data_x = data_x.assign(ps=0)
for train_index, test_index in indices:
    prob_m.fit(X=data_x[X].iloc[train_index], y=data_y['Z'].iloc[train_index].values.ravel())
    data_x.loc[test_index, 'ps'] = prob_m.predict_proba(X=data_x[X].iloc[test_index])[:,1]
    
df = pd.merge(data_x, data_y, on='id.practice')

# year average

In [198]:
# avg for treated and untreated

means = []
for i in range(1,5):
    means.append(np.average(df[(df['year'] == i) & (df['Z'] == 0)]['Y'], weights = df[(df['year'] == i) & (df['Z'] == 0)]['n.patients']))
    print(means[i-1])

avgs = pd.DataFrame({'year': [1, 2, 3, 4], 'ft': means})
df = pd.merge(df, avgs, on='year')

df_34 = df[df['year'] >= 3]

for i in [1,2,3,4]:
    print(np.average(df[(df['year'] == i) & (df['Z'] == 1)]['Y'], weights = df[(df['year'] == i) & (df['Z'] == 1)]['n.patients']))
    

# avg for year 1,2
for i in [1,2]:
    print(np.average(df[(df['year'] == i)]['Y'], weights = df[(df['year'] == i)]['n.patients']))

865.3614330552035
1006.572361414004
1127.312134618135
1247.3123710094226
867.6927807920633
1087.793746374329
1213.996301442881
1339.7549908514943
866.4015254031086
1042.6514616793997


# mean function $g_0(X,V)$

In [126]:
# temporary dataframes
df_0 = df[(df['year'] <= 2) | (df['Z'] == 0)]
df_3 = df[(df['year'] == 3) | (df['Z'] == 1)]
df_4 = df[(df['year'] == 4) | (df['Z'] == 1)]

In [64]:
test_index

array([  1,   7,  22,  37,  50,  68,  71,  72,  86, 115, 129, 133, 141,
       144, 156, 178, 203, 209, 215, 216, 235, 237, 241, 252, 254, 255,
       264, 276, 281, 313, 316, 317, 319, 335, 352, 357, 384, 390, 393,
       395, 396, 398, 402, 413, 431, 448, 460, 465, 468, 494])

In [ ]:
(data_y['year']-data_y['Z']).iloc[test_index]

In [201]:
(df['Y']-df['ft']).shape

(2000,)

In [199]:
# g_0(X,V)
df = df.assign(g0=0)
for train_index, test_index in indices:
    tr = df['Z']
    train = np.concatenate((train_index, train_index+500, train_index+1000, train_index+1500))
    test = np.concatenate((test_index, test_index+500, test_index+1000, test_index+1500))
    xx = df[XV].iloc[train]
    yy = (df['Y']-df['ft']).iloc[train]
    sw = df['n.patients'].iloc[np.concatenate((train_index, train_index+500, train_index+1000, train_index+1500))]
    
    # use all data of year 1,2; untreated groups of year 3,4
    xx = xx[tr == 0]
    yy = yy[tr == 0]
    sw = sw[tr == 0]
    mean_m.fit(X=xx, y=yy, sample_weight=sw)    
    df.loc[test, 'g0'] = mean_m.predict(df[XV].iloc[test])    

C:\Users\wj\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


# ATTEs

In [218]:
psi_3.mean()

44.74772250563123

In [219]:
df_3 = df[df['year'] == 3]
df_4 = df[df['year'] == 4]
df_34 = df[df['year'] >= 3]

psi_3 = df_3['Z'] * (df_3['Y'] - df_3['g0'] - df_3['ft']) - df_3['ps'] / (1 - df_3['ps']) * (1 - df_3['Z']) * (df_3['Y'] - df_3['g0'] - df_3['ft'])
psi_4 = df_4['Z'] * (df_4['Y'] - df_4['g0'] - df_4['ft']) - df_4['ps'] / (1 - df_4['ps']) * (1 - df_4['Z']) * (df_4['Y'] - df_4['g0'] - df_4['ft'])
psi_34 = df_34['Z'] * (df_34['Y'] - df_34['g0'] - df_34['ft']) - df_34['ps'] / (1 - df_34['ps']) * (1 - df_34['Z']) * (df_34['Y'] - df_34['g0'] - df_34['ft'])


tau_3 = np.average(psi_3, weights=df_3['n.patients']) / np.average(df_3['Z'], weights=df_3['n.patients'])
tau_4 = np.average(psi_4, weights=df_4['n.patients']) / np.average(df_4['Z'], weights=df_4['n.patients'])
tau_34 = np.average(psi_34, weights=df_34['n.patients']) / np.average(df_34['Z'], weights=df_34['n.patients'])

# confidence intervals

In [220]:
p3 = np.average(df_3['ps'],  weights=df_3['n.patients'])
p4 = np.average(df_4['ps'],  weights=df_4['n.patients'])
p34 = np.average(df_34['ps'],  weights=df_34['n.patients'])

sigma_3 = psi_3 - df_3['Z'] * tau_3 / p3
sigma_4 = psi_4 - df_4['Z'] * tau_4 / p4
sigma_34 = psi_34 - df_34['Z'] * tau_34 / p34

sigma_3 = (np.average(sigma_3 ** 2, weights=df_3['n.patients'])) ** 0.5
sigma_4 = (np.average(sigma_4 ** 2, weights=df_4['n.patients'])) ** 0.5
sigma_34 = (np.average(sigma_34 ** 2, weights=df_34['n.patients'])) ** 0.5

qt = scipy.stats.norm.ppf(0.95)
ci_3 = (tau_3 - qt * sigma_3 / 500, tau_3 + qt * sigma_3 / 500)
ci_4 = (tau_4 - qt * sigma_4 / 500, tau_4 + qt * sigma_4 / 500)
ci_34 = (tau_34 - qt * sigma_34 / 500, tau_34 + qt * sigma_34 / 500)

In [221]:
print(tau_3, ci_3)
print(tau_4, ci_4)
print(tau_34, ci_34)

19.55408781513193 (17.60754144288514, 21.50063418737872)
44.69332665843247 (42.94081396045842, 46.445839356406516)
32.098507050662 (30.246718529147802, 33.950295572176195)
